In [5]:
# Prelimenary Imports and ENV variable definitions
import csv
import os
import yfinance as yf

from dotenv import load_dotenv
from pprint import pprint

load_dotenv()
FILE_PATH = os.environ.get("FILE_PATH") 


CIK_IDENTIFIERS = [
    '0001720792',
    '0001099281',
    '0001079114',
    '0001112520',
    '0001641864',
    '0000846222',
    '0001709323',
    '0000732905',
    '0000883965',
    '0001067983',
    '0001061768',
]

From the SUBMISSION table fetch a list of ACCESSION_NUMBER(s) using the CIK identifiers in table A-1 (Appendix).




In [6]:

picked_submissions = []
with open(FILE_PATH + 'SUBMISSION_2023_q1.tsv', 'r', encoding='utf-8') as q1:
    for submission in csv.DictReader(q1, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])

with open(FILE_PATH + 'SUBMISSION_2023_q2.tsv', 'r', encoding='utf-8') as q2:
    for submission in csv.DictReader(q2, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])

with open(FILE_PATH + 'SUBMISSION_2023_q3.tsv', 'r', encoding='utf-8') as q3:
    for submission in csv.DictReader(q3, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])
    

pprint(picked_submissions)


['0000950123-23-002585',
 '0001172661-23-001336',
 '0001709323-23-000001',
 '0000919574-23-001107',
 '0001567619-23-002543',
 '0001112520-23-000006',
 '0001213900-23-001958',
 '0001099281-23-000009',
 '0001641864-23-000002',
 '0000950123-23-001270',
 '0001821268-23-000059',
 '0001641864-23-000004',
 '0001099281-23-000012',
 '0001709323-23-000004',
 '0001172661-23-002282',
 '0000919574-23-003116',
 '0000950123-23-005270',
 '0000950123-23-004966',
 '0001821268-23-000090',
 '0001567619-23-006061',
 '0001099281-23-000010',
 '0001641864-23-000003',
 '0001112520-23-000008',
 '0001213900-23-026907',
 '0000919574-23-004553',
 '0001172661-23-003174',
 '0001709323-23-000005',
 '0000950123-23-007916',
 '0000950123-23-008074',
 '0001099281-23-000013',
 '0001213900-23-056683',
 '0001112520-23-000013',
 '0001567619-23-006910',
 '0001821268-23-000158',
 '0001641864-23-000005']


From the INFOTABLE fetch a list of NAMEOFISSUER(s) using the ACCESSION_NUMBER(s) created in (b). Use CUSIP(s) to map between brokers since it is unique where names differ slightly.

In [7]:
names_of_issuers = set()

with open(FILE_PATH + 'INFOTABLE_2023_q1.tsv', 'r', encoding='utf-8') as q1:
    for entry in csv.DictReader(q1, delimiter="\t"):
        if entry["ACCESSION_NUMBER"] in picked_submissions:
            names_of_issuers.add(entry["CUSIP"].upper())
            
with open(FILE_PATH + 'INFOTABLE_2023_q2.tsv', 'r', encoding='utf-8') as q2:
    for entry in csv.DictReader(q2, delimiter="\t"):
        if entry["ACCESSION_NUMBER"] in picked_submissions:
            names_of_issuers.add(entry["CUSIP"].upper())
            
with open(FILE_PATH + 'INFOTABLE_2023_q3.tsv', 'r', encoding='utf-8') as q3:
    for entry in csv.DictReader(q3, delimiter="\t"):
        if entry["ACCESSION_NUMBER"] in picked_submissions:
            names_of_issuers.add(entry["CUSIP"].upper())
            
pprint(names_of_issuers)


{'00183L102',
 '004498101',
 '00507V109',
 '00724F101',
 '00751Y106',
 '00791N102',
 '00846U101',
 '011311107',
 '011642105',
 '01609W102',
 '02005N100',
 '02079K107',
 '02079K305',
 '02156K103',
 '023135106',
 '023586100',
 '023586506',
 '025816109',
 '02665T306',
 '03027X100',
 '031100100',
 '032654105',
 '036752103',
 '037833100',
 '040413106',
 '042735100',
 '046513107',
 '05278C107',
 '052800109',
 '053332102',
 '053807103',
 '056752108',
 '060505104',
 '064058100',
 '06417N103',
 '071813109',
 '073463119',
 '084670108',
 '084670702',
 '090040AF3',
 '09215C105',
 '09263B116',
 '099406100',
 '10922N103',
 '11271J107',
 '113004105',
 '12008R107',
 '12504L109',
 '12532H104',
 '12562N104',
 '12626K203',
 '12653C108',
 '14040H105',
 '142339100',
 '143130102',
 '150870103',
 '155123102',
 '16119P108',
 '16150R104',
 '166764100',
 '171779309',
 '17275R102',
 '172967424',
 '17888H103',
 '191216100',
 '191241108',
 '19247G107',
 '19260QAB3',
 '199908104',
 '20030N101',
 '200340107',
 '2060

Now we need to convert the CUSIP to tickers, we will do this using the polygon API to fetch info about a holding by it's CUSIP ID. 

Simply download the last 2 most recent file from https://www.sec.gov/data/foiadocsfailsdatahtm and store in dataset folder.

In this step we lose about 12% of the dataset... Unsure if there is a better way to resolve this

In [8]:
tickers = set()
with open(FILE_PATH + 'cnsfails202311a','r') as f:
    for entry in csv.DictReader(f, delimiter="|"):
        if entry['CUSIP'] in names_of_issuers: 
            tickers.add(entry['SYMBOL'])
            names_of_issuers.remove(entry['CUSIP'])
with open(FILE_PATH + 'cnsfails202310b','r') as f:
    for entry in csv.DictReader(f, delimiter="|"):
        if entry['CUSIP'] in names_of_issuers: 
            tickers.add(entry['SYMBOL'])
            names_of_issuers.remove(entry['CUSIP'])
    
pprint(tickers)

{'A',
 'AAP',
 'AAPL',
 'ACIW',
 'ACN',
 'ADBE',
 'ADI',
 'ADV',
 'ALG',
 'ALLY',
 'ALRM',
 'ALV',
 'AME',
 'AMH',
 'AMT',
 'AMZN',
 'ANET',
 'ANGI',
 'AON',
 'ARW',
 'ASAI',
 'ATAI',
 'ATHM',
 'ATRA',
 'ATUS',
 'ATVI',
 'AVT',
 'AXP',
 'AXTA',
 'AZO',
 'BABA',
 'BAC',
 'BAM',
 'BAX',
 'BHF',
 'BIDU',
 'BIL',
 'BK',
 'BKSYWS',
 'BLDR',
 'BN',
 'BODYWS',
 'BOOT',
 'BRKB',
 'C',
 'CACC',
 'CBRE',
 'CCF',
 'CCRD',
 'CE',
 'CEIX',
 'CET',
 'CHTR',
 'CIEN',
 'CIVI',
 'CKX',
 'CMA',
 'CMCSA',
 'CNHI',
 'CNX',
 'CNXC',
 'COF',
 'COHR',
 'COP',
 'COST',
 'COYA',
 'CPRI',
 'CPRT',
 'CRM',
 'CSCO',
 'CSGP',
 'CSL',
 'CTO',
 'CVX',
 'DB',
 'DBRG',
 'DEO',
 'DG',
 'DHI',
 'DHR',
 'DIS',
 'DLB',
 'DLTR',
 'DNMR',
 'DVA',
 'ECPG',
 'EDU',
 'ELV',
 'EME',
 'EMR',
 'EPD',
 'EQC',
 'EQH',
 'EW',
 'EWBC',
 'F',
 'FAST',
 'FBIN',
 'FCG',
 'FCNCA',
 'FDX',
 'FHN',
 'FI',
 'FIS',
 'FIVE',
 'FIX',
 'FMC',
 'FMS',
 'FNCH',
 'FND',
 'FNF',
 'FNKO',
 'FOX',
 'FR',
 'FRCB',
 'FRPH',
 'FTI',
 'FTV',
 'GANX',
 'G

Adding the rest of the ticker symbols to the set from the other datasets.

Before anything delete the frist 10 rows of the csv files {DIVB_holdings, HDV_holdings} as it messed up the parsing for DictReader.

In [9]:
with open(FILE_PATH + 'DIVB_holdings.csv','r', encoding='utf-8-sig') as f:
    for entry in csv.DictReader(f, delimiter=","):
        entry.keys()
        tickers.add(entry["Ticker"])
        
with open(FILE_PATH + 'HDV_holdings.csv','r', encoding='utf-8-sig') as f:
    for entry in csv.DictReader(f, delimiter=","):
        tickers.add(entry["Ticker"])

pprint(tickers)

{'A',
 'AAP',
 'AAPL',
 'ABBV',
 'ACIW',
 'ACN',
 'ACT',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADT',
 'ADV',
 'AEE',
 'AEO',
 'AEP',
 'AES',
 'AFL',
 'AGNC',
 'AIG',
 'AIRC',
 'AIZ',
 'ALE',
 'ALG',
 'ALL',
 'ALLY',
 'ALRM',
 'ALSN',
 'ALV',
 'AM',
 'AMCR',
 'AME',
 'AMGN',
 'AMH',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANGI',
 'AON',
 'AOS',
 'APA',
 'APAM',
 'APD',
 'APLE',
 'APO',
 'ARCH',
 'ARES',
 'ARW',
 'ASAI',
 'ASB',
 'ASH',
 'ATAI',
 'ATHM',
 'ATRA',
 'ATUS',
 'ATVI',
 'AUB',
 'AVB',
 'AVGO',
 'AVNT',
 'AVT',
 'AXP',
 'AXS',
 'AXTA',
 'AZO',
 'BABA',
 'BAC',
 'BAM',
 'BAX',
 'BC',
 'BEN',
 'BG',
 'BHF',
 'BIDU',
 'BIL',
 'BK',
 'BKH',
 'BKR',
 'BKSYWS',
 'BLDR',
 'BLK',
 'BLMN',
 'BMY',
 'BN',
 'BODYWS',
 'BOKF',
 'BOOT',
 'BPOP',
 'BRKB',
 'BRX',
 'BX',
 'BXMT',
 'BXP',
 'C',
 'CABO',
 'CACC',
 'CADE',
 'CAG',
 'CAH',
 'CALM',
 'CAT',
 'CATY',
 'CB',
 'CBRE',
 'CBRL',
 'CBSH',
 'CC',
 'CCF',
 'CCI',
 'CCK',
 'CCOI',
 'CCRD',
 'CDP',
 'CE',
 'CEIX',
 'CET',
 'CF',
 'CFG',
 'CFR',
 

From set A, remove all tickers that do not offer dividends

In [10]:
from requests import HTTPError


arr_A = list(tickers)

ticker_objs = list(yf.Tickers(arr_A).tickers.values())
arr_B = []
for ticker in ticker_objs:
    try:
        if 'dividendRate' in ticker.info.keys():
            arr_B.append(ticker.info["symbol"])
    except HTTPError:
        print(f"Ticker not found, removed from subset.")
        continue

pprint(arr_B)

Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
Ticker not found, removed from subset.
['RTX',
 'ONB',
 'LIN',
 'GEN',
 'CTRA',
 'ASH',
 'AMGN',
 'WOR',
 'UNH',
 'FITB',
 'WSM',
 'APLE',
 'ROIC',
 'DVN',
 'RGA',
 'UMBF',
 'EBC',
 'DBRG',
 'HLN',
 'PHM',
 'CNHI',
 'TGT',
 'ORCL',
 'EPD',
 'SCI',
 'CMCSA',
 'LKQ',
 'OC',
 'DAN',
 'CCOI',
 'EMN',
 'BABA',
 'HPE',
 'CRK',
 'VTR',
 'TFC',
 'NFE',
 'EQH',
 'CSL',
 'KMB',
 'FOXA',
 'AIRC',
 'HCC',
 'TNL',
 'A',
 'EXC',
 'HON',
 'MAC',
 'COP',
 'MPW',
 'CCK',
 'HAS',
 'MS',
 'EIX',
 'GIL',
 'D

From subset (b) remove all names that have a high business risk, a debt to equity ratio greater than 1.5, sub-subset (c)

In [28]:
ticker_objs = list(yf.Tickers(arr_B).tickers.values())


arr_C = []

for ticker in ticker_objs:
    balance_sheet = list(ticker.balancesheet.to_dict().values())[0] # get most recent data
    liabilities = balance_sheet['Total Liabilities Net Minority Interest']
    assets = balance_sheet['Total Assets']
    try:
        debtToEquity = abs( liabilities / (assets - liabilities) )
    except ZeroDivisionError:
        print(ticker.info["symbol"]) # if this is close to 0 then equity to debt ratio is near inf 
        continue                     # So we skip it.
    if debtToEquity <= 1.5:
        arr_C.append(ticker.info["symbol"])

print(len(arr_C))
pprint(arr_C)
    



MCO
PFG
FAF
AEP
CVX
210
['RTX',
 'LIN',
 'CTRA',
 'ASH',
 'WOR',
 'APLE',
 'ROIC',
 'DVN',
 'DBRG',
 'HLN',
 'PHM',
 'EPD',
 'LKQ',
 'OC',
 'BABA',
 'CRK',
 'VTR',
 'CSL',
 'FOXA',
 'HCC',
 'A',
 'COP',
 'MPW',
 'GIL',
 'DTM',
 'MTG',
 'FHI',
 'NHI',
 'HIW',
 'HLI',
 'MGY',
 'BX',
 'FANG',
 'INGR',
 'FAST',
 'WY',
 'CCF',
 'CNXC',
 'PK',
 'SITC',
 'SPGI',
 'CSCO',
 'MKTX',
 'SBRA',
 'TOL',
 'PARAA',
 'ICFI',
 'HST',
 'EPR',
 'ARCH',
 'CNS',
 'STC',
 'PLD',
 'MATX',
 'UNF',
 'LXP',
 'DEI',
 'KMT',
 'CIVI',
 'IVT',
 'IR',
 'ESNT',
 'PTEN',
 'PXD',
 'VTS',
 'KIM',
 'TAP',
 'ITIC',
 'DLB',
 'TSM',
 'SNA',
 'LHX',
 'IVZ',
 'CHRD',
 'ALG',
 'INVH',
 'DHI',
 'ACT',
 'PSA',
 'DINO',
 'DPZ',
 'PKG',
 'MLM',
 'TECK',
 'RMD',
 'JCI',
 'KRG',
 'FR',
 'ESS',
 'DD',
 'KOF',
 'SUI',
 'MOS',
 'ATHM',
 'MUR',
 'ALE',
 'SWKS',
 'MRO',
 'MAA',
 'RDN',
 'PARA',
 'MRK',
 'GOLF',
 'CDP',
 'SEB',
 'NNN',
 'MDC',
 'LPX',
 'J',
 'TXN',
 'DIS',
 'LCII',
 'LEN',
 'AOS',
 'ODFL',
 'BEN',
 'LH',
 'ADM',
 'GPN',
 '